In [4]:
import cv2
import mediapipe as mp
import pyttsx3

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

# Initialize text-to-speech engine with female voice
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)  # Assuming the female voice is at index 1

def detectPose(frame, pose_model, display=True):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_model.process(rgb_frame)
    
    if results.pose_landmarks:
        if display:
            mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
        return True, frame, results.pose_landmarks
    else:
        return False, frame, None

# Start video capture
video = cv2.VideoCapture(0)
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)

def speak(message):
    engine.say(message)
    engine.runAndWait()

speak("Please stand in front of the camera so that all body parts can be detected.")

while video.isOpened():
    ok, frame = video.read()
    if not ok:
        break
    
    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
    detected, frame, landmarks = detectPose(frame, pose_video, display=True)
    
    if not detected:
        missing_parts = []
        if landmarks is not None:
            # Check for missing landmarks
            if landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST] is None:
                missing_parts.append('right hand')
            if landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST] is None:
                missing_parts.append('left hand')
            if landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE] is None:
                missing_parts.append('right leg')
            if landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE] is None:
                missing_parts.append('left leg')
        
        if missing_parts:
            speak(f"I can't see your {', '.join(missing_parts)}. Please adjust your position.")
        else:
            speak("All body parts are detected. You are ready.")

    cv2.imshow('Pose Detection', frame)

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

video.release()
cv2.destroyAllWindows()
